# Healthcare Field Mappings Setup

This section initializes the `mappings_list` table with standard healthcare and insurance field names. The table serves as a reference for field mapping and data standardization.

## Table Structure
- **src**: Source field name
- **target**: Target field name (standardized)

## Field Categories Included:

### Policy & Member Information
- Policy Effective Date
- Group Name
- Insured First Name, Last Name, DOB
- Claimant First Name, Last Name, DOB

### Service & Claims Data
- Beginning/Ending Service Date
- Processed Date
- Claim Number/Claim Control Number
- Service Line/Claim Type

### Medical Coding
- Primary ICD, Secondary ICD
- CPT Code, HCPCS Code
- Revenue Code, Modifier Code

### Prescription Information
- Rx Name, Quantity, Days Supply
- Rx Date Filled

### Financial Amounts
- Billed Amount, Allowed Amount, Paid Amount
- Copay, Deductible, Coinsurance Amount
- Net, Ineligible, COB, Other Reduced, Denied Amount

### Provider Information
- NPI (National Provider Identifier)
- Payee Name, Address, TIN

## Usage
This mapping table will be used by the FieldMatcher model to automatically match incoming field names to standardized target field names, enabling consistent data processing across different healthcare data sources.

In [ ]:
--DROP TABLE mappings_list; --If you want to start from scratch
CREATE TABLE IF NOT EXISTS mappings_list (
    src VARCHAR,
    target VARCHAR
);

-- Use MERGE to avoid duplicates
MERGE INTO mappings_list AS target
USING (
    VALUES
    ('Policy Effective Date', 'Policy Effective Date'),
    ('Group Name', 'Group Name'),
    ('Insured First Name', 'Insured First Name'),
    ('Insured Last Name', 'Insured Last Name'),
    ('Insured DOB', 'Insured DOB'),
    ('Claimant First Name', 'Claimant First Name'),
    ('Claimant Last Name', 'Claimant Last Name'),
    ('Claimant DOB', 'Claimant DOB'),
    ('Beginning Service Date', 'Beginning Service Date'),
    ('Claim Number/Claim Control Number', 'Claim Number/Claim Control Number'),
    ('Ending Service Date', 'Ending Service Date'),
    ('Processed Date', 'Processed Date'),
    ('Primary ICD', 'Primary ICD'),
    ('Secondary ICD', 'Secondary ICD'),
    ('CPT Code', 'CPT Code'),
    ('HCPCS Code', 'HCPCS Code'),
    ('Revenue Code', 'Revenue Code'),
    ('Modifier Code', 'Modifier Code'),
    ('Product Type', 'Product Type'),
    ('NPI?', 'NPI?'),
    ('Rx Name', 'Rx Name'),
    ('Rx Quantity', 'Rx Quantity'),
    ('Rx Days Supply', 'Rx Days Supply'),
    ('Rx Date Filled', 'Rx Date Filled'),
    ('Billed Amount', 'Billed Amount'),
    ('Copay Amount', 'Copay Amount'),
    ('Deductible Amount', 'Deductible Amount'),
    ('Coinsurance Amount', 'Coinsurance Amount'),
    ('Allowed Amount', 'Allowed Amount'),
    ('Net Amount', 'Net Amount'),
    ('Ineligible Amount', 'Ineligible Amount'),
    ('COB Amount', 'COB Amount'),
    ('Other Reduced Amount', 'Other Reduced Amount'),
    ('Denied Amount', 'Denied Amount'),
    ('Paid Amount', 'Paid Amount'),
    ('Service Line/Claim Type', 'Service Line/Claim Type'),
    ('Payee Name', 'Payee Name'),
    ('Payee Address', 'Payee Address'),
    ('Payee TIN', 'Payee TIN')
) AS source (src, target)
ON target.src = source.src AND target.target = source.target
WHEN NOT MATCHED THEN
    INSERT (src, target) VALUES (source.src, source.target);


In [ ]:
# Healthcare Field Matching System - CORRECTED VERSION
# This module provides intelligent field name matching using multiple similarity algorithms

from snowflake.snowpark.context import get_active_session
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from difflib import SequenceMatcher
import re

# Initialize Snowpark session for database connectivity
session = get_active_session()

# Load the mappings data from Snowflake table
mappings_df = session.sql("SELECT * FROM mappings_list").to_pandas()

class FieldMatcher:
    """
    Healthcare Field Matching System
    
    This class provides intelligent field name matching using multiple similarity algorithms
    to map incoming field names to standardized healthcare field names.
    
    Features:
    - Exact matching for identical field names
    - Substring matching for partial matches
    - Sequence similarity using difflib for character-level similarity
    - Word overlap using Jaccard similarity coefficient
    - TF-IDF vectorization with cosine similarity for semantic matching
    
    Attributes:
        mappings_df (DataFrame): Pandas DataFrame containing source and target field mappings
        src_fields (list): List of source field names to match against
        target_fields (list): List of standardized target field names
        src_to_target (dict): Dictionary mapping source fields to target fields
        tfidf (TfidfVectorizer): Scikit-learn TF-IDF vectorizer for semantic analysis
        tfidf_matrix: Fitted TF-IDF matrix of source field names
    """
    
    def __init__(self, mappings_df):
        """
        Initialize the FieldMatcher with mapping data
        
        Args:
            mappings_df (DataFrame): DataFrame with 'SRC' and 'TARGET' columns
        """
        self.mappings_df = mappings_df
        # Match against source fields, not target fields
        self.src_fields = mappings_df['SRC'].tolist()
        self.target_fields = mappings_df['TARGET'].tolist()
        
        # Create mapping dictionary for quick lookup
        self.src_to_target = dict(zip(mappings_df['SRC'], mappings_df['TARGET']))
        
        # Initialize TF-IDF vectorizer for semantic similarity
        # Uses 1-3 word n-grams to capture multi-word field names
        self.tfidf = TfidfVectorizer(
            lowercase=True,           # Convert all text to lowercase
            ngram_range=(1, 3),      # Use 1-3 word combinations for better matching
            max_features=1000        # Limit vocabulary size for performance
        )
        
        # Pre-fit vectorizer on source fields (not target fields)
        self.tfidf_matrix = self.tfidf.fit_transform(self.src_fields)
    
    def preprocess_text(self, text):
        """
        Clean and normalize text for consistent matching
        
        This function standardizes input text by:
        - Converting to lowercase
        - Removing special characters and punctuation
        - Normalizing whitespace
        
        Args:
            text (str): Input text to preprocess
            
        Returns:
            str: Cleaned and normalized text
        """
        if not text:
            return ""
        # Convert to lowercase and remove special characters (keep only alphanumeric and spaces)
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        # Remove extra whitespace and normalize spacing
        text = ' '.join(text.split())
        return text
    
    def calculate_similarity_scores(self, input_text):
        """
        Calculate multiple similarity scores between input text and all source fields
        
        This method computes four different similarity metrics:
        1. Exact Match: Binary score for identical strings after preprocessing
        2. Substring Match: Binary score if one string contains the other
        3. Sequence Similarity: Character-level similarity using difflib
        4. Word Overlap: Jaccard similarity coefficient of word sets
        
        Args:
            input_text (str): Field name to match against source fields
            
        Returns:
            list: List of dictionaries containing scores for each source field
        """
        input_text = self.preprocess_text(input_text)
        scores = []
        
        # Compare against source fields, not target fields
        for src_field in self.src_fields:
            src_clean = self.preprocess_text(src_field)
            
            # 1. Exact match score - highest confidence when strings are identical
            exact_score = 1.0 if input_text == src_clean else 0.0
            
            # 2. Substring match score - high confidence for partial matches
            substring_score = 1.0 if input_text in src_clean or src_clean in input_text else 0.0
            
            # 3. Sequence similarity using difflib - handles typos and variations
            sequence_score = SequenceMatcher(None, input_text, src_clean).ratio()
            
            # 4. Word overlap score using Jaccard similarity
            # Measures overlap between sets of words in each field name
            input_words = set(input_text.split())
            src_words = set(src_clean.split())
            if len(input_words.union(src_words)) > 0:
                word_overlap = len(input_words.intersection(src_words)) / len(input_words.union(src_words))
            else:
                word_overlap = 0.0
            
            # Combine scores with weighted average
            # Weights: Exact (40%), Substring (20%), Sequence (20%), Word Overlap (20%)
            combined_score = (
                exact_score * 0.4 +
                substring_score * 0.2 +
                sequence_score * 0.2 +
                word_overlap * 0.2
            )
            
            # Store all scores for analysis and debugging
            # Return the corresponding target field for the matched source field
            scores.append({
                'src_field': src_field,
                'target_field': self.src_to_target[src_field],
                'combined_score': combined_score,
                'exact_score': exact_score,
                'substring_score': substring_score,
                'sequence_score': sequence_score,
                'word_overlap': word_overlap
            })
        
        return scores
    
    def predict_match(self, input_text, top_n=3, min_threshold=0.1):
        """
        Predict the best matching target field(s) for input text
        Matches against SRC fields but returns TARGET field names
        
        This method combines multiple similarity algorithms with TF-IDF semantic matching
        to provide ranked predictions with confidence scores.
        
        Args:
            input_text (str): Field name to match
            top_n (int): Maximum number of predictions to return (default: 3)
            min_threshold (float): Minimum score threshold for predictions (default: 0.1)
            
        Returns:
            list: Sorted list of prediction dictionaries with scores and field names
        """
        # Get basic similarity scores (comparing against SRC fields)
        scores = self.calculate_similarity_scores(input_text)
        
        # Add TF-IDF cosine similarity for semantic matching (against SRC fields)
        # This helps match fields with similar meaning but different wording
        input_vector = self.tfidf.transform([self.preprocess_text(input_text)])
        cosine_scores = cosine_similarity(input_vector, self.tfidf_matrix)[0]
        
        # Enhance combined scores with TF-IDF semantic similarity
        for i, score_dict in enumerate(scores):
            score_dict['tfidf_score'] = cosine_scores[i]
            # Update combined score: 70% basic similarity + 30% semantic similarity
            score_dict['combined_score'] = (
                score_dict['combined_score'] * 0.7 + 
                cosine_scores[i] * 0.3
            )
        
        # Sort by combined score (highest confidence first)
        scores = sorted(scores, key=lambda x: x['combined_score'], reverse=True)
        
        # Filter by minimum threshold and return top N predictions
        filtered_scores = [s for s in scores if s['combined_score'] >= min_threshold]
        
        return filtered_scores[:top_n]
    
    def batch_predict(self, input_list, top_n=1):
        """
        Predict matches for multiple input values efficiently
        
        This method processes a list of field names and returns the best match
        for each input along with confidence scores.
        
        Args:
            input_list (list): List of field names to match
            top_n (int): Number of predictions per input (default: 1 for best match only)
            
        Returns:
            list: List of prediction results with input, predicted target, and confidence
        """
        results = []
        for input_text in input_list:
            predictions = self.predict_match(input_text, top_n=top_n)
            # Get best match or indicate no match found
            best_match = predictions[0] if predictions else {'target_field': 'NO_MATCH', 'combined_score': 0.0}
            results.append({
                'input': input_text,
                'predicted_target': best_match['target_field'],
                'confidence_score': best_match['combined_score']
            })
        return results

In [ ]:

# Initialize the matcher
matcher = FieldMatcher(mappings_df)

# Test the model with some examples
test_inputs = [
    "amt",
    "Policy Effective Date",
    "Policy Start Date",
    "Member First Name", 
    "Patient DOB",
    "Claim ID",
    "Service Begin Date",
    "Diagnosis Code",
    "Payment Amount",
    "Provider NPI"
]

print("Testing the Field Matching Model:")
print("=" * 50)

for test_input in test_inputs:
    predictions = matcher.predict_match(test_input, top_n=3)
    print(f"\nInput: '{test_input}'")
    print("Top 3 Predictions:")
    
    if predictions:
        for i, pred in enumerate(predictions, 1):
            print(f"  {i}. {pred['target_field']} (Score: {pred['combined_score']:.3f})")
    else:
        print("  No good matches found")

# Create a function for easy prediction
def predict_field_mapping(input_text):
    """Simple function to get the best field mapping prediction"""
    predictions = matcher.predict_match(input_text, top_n=1)
    if predictions and predictions[0]['combined_score'] > 0.3:  # Confidence threshold
        return predictions[0]['target_field']
    else:
        return "NO_CONFIDENT_MATCH"

print(f"\n\nQuick prediction examples:")
print(f"'Patient Name' -> {predict_field_mapping('Patient Name')}")
print(f"'Birth Date' -> {predict_field_mapping('Birth Date')}")
print(f"'Claim Control Number' -> {predict_field_mapping('Claim Control Number')}")

In [ ]:
-- Corrected Field Matcher Procedure - Matches SRC fields, Returns TARGET fields
CREATE OR REPLACE PROCEDURE field_matcher_advanced(
    input_fields ARRAY,
    top_n INTEGER DEFAULT 3,
    min_threshold FLOAT DEFAULT 0.1
)
RETURNS TABLE (
    input_field STRING,
    src_field STRING,
    target_field STRING,
    combined_score FLOAT,
    exact_score FLOAT,
    substring_score FLOAT,
    sequence_score FLOAT,
    word_overlap FLOAT,
    tfidf_score FLOAT,
    match_rank INTEGER
)
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('numpy', 'scikit-learn', 'snowflake-snowpark-python')
HANDLER = 'run'
AS
$$
import re
import numpy as np
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import StructType, StructField, StringType, FloatType, IntegerType

def preprocess_text(text):
    """Clean and normalize text for better matching"""
    if not text:
        return ""
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    return ' '.join(text.split())

def calculate_similarity_scores(input_text, src_fields, src_to_target_map):
    """Calculate various similarity scores for the input text against all source fields"""
    input_clean = preprocess_text(input_text)
    scores = []
    
    for src_field in src_fields:
        src_clean = preprocess_text(src_field)
        
        # 1. Exact match score
        exact_score = 1.0 if input_clean == src_clean else 0.0
        
        # 2. Substring match score
        substring_score = 1.0 if (input_clean in src_clean or src_clean in input_clean) else 0.0
        
        # 3. Sequence similarity (difflib)
        sequence_score = SequenceMatcher(None, input_clean, src_clean).ratio()
        
        # 4. Word overlap score
        input_words = set(input_clean.split()) if input_clean else set()
        src_words = set(src_clean.split()) if src_clean else set()
        
        if len(input_words.union(src_words)) > 0:
            word_overlap = len(input_words.intersection(src_words)) / len(input_words.union(src_words))
        else:
            word_overlap = 0.0
        
        # Initial combined score (before TF-IDF)
        combined_score = (
            exact_score * 0.4 +
            substring_score * 0.2 +
            sequence_score * 0.2 +
            word_overlap * 0.2
        )
        
        scores.append({
            'src_field': src_field,
            'target_field': src_to_target_map.get(src_field, 'UNMAPPED'),
            'combined_score': combined_score,
            'exact_score': exact_score,
            'substring_score': substring_score,
            'sequence_score': sequence_score,
            'word_overlap': word_overlap,
            'tfidf_score': 0.0  # Will be updated later
        })
    
    return scores

def add_tfidf_scores(input_text, scores, src_fields):
    """Add TF-IDF cosine similarity to the scores"""
    try:
        # Initialize TF-IDF vectorizer
        tfidf = TfidfVectorizer(
            lowercase=True,
            ngram_range=(1, 3),
            max_features=1000
        )
        
        # Prepare text data - preprocess source fields
        preprocessed_src_fields = [preprocess_text(field) for field in src_fields]
        preprocessed_input = preprocess_text(input_text)
        
        # Fit vectorizer on source fields (not target fields)
        tfidf_matrix = tfidf.fit_transform(preprocessed_src_fields)
        
        # Transform input text
        input_vector = tfidf.transform([preprocessed_input])
        
        # Calculate cosine similarity
        cosine_scores = cosine_similarity(input_vector, tfidf_matrix)[0]
        
        # Update scores with TF-IDF
        for i, score_dict in enumerate(scores):
            score_dict['tfidf_score'] = float(cosine_scores[i])
            # Update combined score: 70% basic similarity + 30% semantic similarity
            score_dict['combined_score'] = (
                score_dict['combined_score'] * 0.7 + 
                cosine_scores[i] * 0.3
            )
            
    except Exception as e:
        # If TF-IDF fails, use original scores
        for score_dict in scores:
            score_dict['tfidf_score'] = 0.0
    
    return scores

def run(session, input_fields, top_n, min_threshold):
    # Get both SRC and TARGET fields from mappings table
    mappings_query = """
    SELECT SRC, TARGET FROM mappings_list ORDER BY SRC
    """
    
    # Execute the query and get results
    mappings_result = session.sql(mappings_query).collect()
    
    # Create lists and mapping dictionary
    src_fields = [row[0] for row in mappings_result]
    src_to_target_map = {row[0]: row[1] for row in mappings_result}
    
    results = []
    
    # Process each input field
    for input_field in input_fields:
        # Calculate similarity scores against SOURCE fields
        field_scores = calculate_similarity_scores(input_field, src_fields, src_to_target_map)
        
        # Add TF-IDF scores (calculated against SOURCE fields)
        field_scores = add_tfidf_scores(input_field, field_scores, src_fields)
        
        # Sort by combined score
        field_scores = sorted(field_scores, key=lambda x: x['combined_score'], reverse=True)
        
        # Filter by minimum threshold and get top N
        filtered_scores = [s for s in field_scores if s['combined_score'] >= min_threshold][:top_n]
        
        # Add to results with ranking
        for rank, score_dict in enumerate(filtered_scores, 1):
            results.append([
                input_field,                                    # input_field
                score_dict['src_field'],                       # src_field (what was matched)
                score_dict['target_field'],                    # target_field (what gets returned)
                round(score_dict['combined_score'], 4),        # combined_score
                round(score_dict['exact_score'], 4),           # exact_score
                round(score_dict['substring_score'], 4),       # substring_score
                round(score_dict['sequence_score'], 4),        # sequence_score
                round(score_dict['word_overlap'], 4),          # word_overlap
                round(score_dict['tfidf_score'], 4),           # tfidf_score
                rank                                           # match_rank
            ])
    
    # Define schema for the DataFrame
    schema = StructType([
        StructField("input_field", StringType()),
        StructField("src_field", StringType()),
        StructField("target_field", StringType()),
        StructField("combined_score", FloatType()),
        StructField("exact_score", FloatType()),
        StructField("substring_score", FloatType()),
        StructField("sequence_score", FloatType()),
        StructField("word_overlap", FloatType()),
        StructField("tfidf_score", FloatType()),
        StructField("match_rank", IntegerType())
    ])
    
    # Create and return Snowpark DataFrame
    return session.create_dataframe(results, schema)
$$;

In [ ]:
-- Test the advanced procedure with multiple fields
CALL field_matcher_advanced(
    ['amt_allowed', 'Patient Name', 'Birth Date', 'Policy Start Date', 'total_amt', 'Diagnosis Code'], 
    3, 
    0.1
);

In [ ]:
-- Simplified version that takes target fields as parameter
CREATE OR REPLACE FUNCTION field_matcher_predict_simple(
    input_field_name STRING,
    target_fields_array ARRAY,
    min_threshold FLOAT DEFAULT 0.3
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
HANDLER = 'run'
AS
$$
import re
from difflib import SequenceMatcher

def preprocess_text(text):
    if not text:
        return ""
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    return ' '.join(text.split())

def calculate_combined_score(input_text, target_text):
    input_clean = preprocess_text(input_text)
    target_clean = preprocess_text(target_text)
    
    # Basic similarity scores
    exact_score = 1.0 if input_clean == target_clean else 0.0
    substring_score = 1.0 if (input_clean in target_clean or target_clean in input_clean) else 0.0
    sequence_score = SequenceMatcher(None, input_clean, target_clean).ratio()
    
    input_words = set(input_clean.split()) if input_clean else set()
    target_words = set(target_clean.split()) if target_clean else set()
    
    if len(input_words.union(target_words)) > 0:
        word_overlap = len(input_words.intersection(target_words)) / len(input_words.union(target_words))
    else:
        word_overlap = 0.0
    
    return (exact_score * 0.4 + substring_score * 0.2 + 
            sequence_score * 0.2 + word_overlap * 0.2)

def run(input_field_name, target_fields_array, min_threshold):
    best_score = 0.0
    best_match = "NO_MATCH"
    
    # Find best match using basic similarity
    for target_field in target_fields_array:
        score = calculate_combined_score(input_field_name, target_field)
        if score > best_score and score >= min_threshold:
            best_score = score
            best_match = target_field
    
    return best_match
$$;

In [ ]:
-- Test simple function with target fields array
WITH target_list AS (
    SELECT ARRAY_AGG(DISTINCT TARGET) as targets 
    FROM mappings_list
)
SELECT 
    field_matcher_predict_simple('Patient First Name', targets, 0.2) as match1,
    field_matcher_predict_simple('DOB', targets, 0.2) as match2,
    field_matcher_predict_simple('Claim ID', targets, 0.2) as match3
FROM target_list;